# Scenario similarity measures

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from matplotlib2tikz import save as tikz_save
import os
%matplotlib inline

In [ ]:
tikz_path = 'figures'
def tikz(name, extras=None):
    # Convert to tikz
    tikz_save(os.path.join(tikz_path, '{:s}.tikz'.format(name)),
              figureheight='\\figureheight', figurewidth='\\figurewidth',
              extra_axis_parameters=extras, show_info=False) 

In [ ]:
# Define function for quickly making tables


## Create two example figures

In [ ]:
with open(os.path.join('..', '20171126 Parametrization', 'df.p'), 'rb') as f:
    (dfs, scaling) = pickle.load(f)
# Only use first n profiles
n = 400
dfs = dfs[:n].copy()
scaling = scaling.T[:n].copy()

In [ ]:
# Show a profile
# 322: hardest braking
# 292: Looks most at 322 when only looking at start and end velocity
# 143: Looks very similar to 292
ii = [322, 143, 292]
ts = 0.1
V = []
A = []
for j, i in enumerate(ii):
    t = dfs[i]['time']*scaling[i, 0]
    v = (dfs[i]['vel']*(scaling[i, 1] - scaling[i, 2]) + scaling[i, 2])*3.6
    tnew = np.arange(t.values[0], np.floor(t.values[-1] / ts)) * ts
    vnew = np.interp(tnew, t, v)
    V.append(vnew)
    A.append(np.gradient(vnew) / ts)
    plt.plot(tnew, vnew, '.', label="Profile {:d}".format(j+1), ms=1)
    print("Average deceleration: {:.2f} m/s^2".format((scaling[i, 1] - scaling[i, 2]) / scaling[i, 0]))
plt.grid('on')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [km/h]')
plt.legend()
tikz('velocity_profile_example')

## Euclidean distance

In [ ]:
def lp(xx, yy, p):
    # Sample yy to have the same number of elements
    yy = np.interp(np.linspace(0, len(yy), len(xx)), np.arange(0, len(yy)), yy)
    
    # Compute the norm
    return np.sum(np.abs(xx - yy)**p) ** (1/p)

def euclidean(xx, yy):
    return lp(xx, yy, 2)

In [ ]:
n = len(V)
for i in range(n):
    for j in range(n):
        if not i == j:
            print("Deuclidean({:d},{:d}) = {:.1f}".format(i+1, j+1, euclidean(V[i], V[j])))

In [ ]:
for i, j in zip([1, 1, 0], [0, 2, 2]):
    f, ax = plt.subplots(1, 1, figsize=(4, 3))
    ax.plot(V[i], '.', label='Profile {:d}'.format(i+1), ms=1)
    ax.plot(np.interp(np.linspace(0, len(V[j]), len(V[i])), np.arange(0, len(V[j])), V[j]), '.', 
            label='Profile {:d}'.format(j+1), ms=1)
    ax.set_xlabel('Sample')
    ax.set_ylabel('Velocity [km/h]')
    ax.grid('on')
    tikz('velocity_profiles_resampled{:d}{:d}'.format(i+1, j+1))